## Imports and Set up


In [155]:
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


number = 1745177083

directory = "../data/" + str(number)
files = os.listdir(directory)
dfs = []
print(files)

# i = 0
for f in files:
    p = os.path.join(directory, f)
    d = pd.read_csv(p)
    dfs.append(d)
    # i += 1
    # if i > 10:
    #     break

df = pd.concat(dfs, ignore_index=True)


Using device: cpu
['spd1vis1sep1.csv', 'spd1vis1sep3.csv', 'spd1vis1sep5.csv', 'spd1vis1sep7.csv', 'spd1vis1sep9.csv', 'spd1vis5sep1.csv', 'spd1vis5sep3.csv', 'spd1vis5sep5.csv', 'spd1vis5sep7.csv', 'spd1vis5sep9.csv', 'spd1vis9sep1.csv', 'spd1vis9sep3.csv', 'spd1vis9sep5.csv', 'spd1vis9sep7.csv', 'spd1vis9sep9.csv', 'spd1vis13sep1.csv', 'spd1vis13sep3.csv', 'spd1vis13sep5.csv', 'spd1vis13sep7.csv', 'spd1vis13sep9.csv', 'spd1vis17sep1.csv', 'spd1vis17sep3.csv', 'spd1vis17sep5.csv', 'spd1vis17sep7.csv', 'spd1vis17sep9.csv', 'spd3vis1sep1.csv', 'spd3vis1sep3.csv', 'spd3vis1sep5.csv', 'spd3vis1sep7.csv', 'spd3vis1sep9.csv', 'spd3vis5sep1.csv', 'spd3vis5sep3.csv', 'spd3vis5sep5.csv', 'spd3vis5sep7.csv', 'spd3vis5sep9.csv', 'spd3vis9sep1.csv', 'spd3vis9sep3.csv', 'spd3vis9sep5.csv', 'spd3vis9sep7.csv', 'spd3vis9sep9.csv', 'spd3vis13sep1.csv', 'spd3vis13sep3.csv', 'spd3vis13sep5.csv', 'spd3vis13sep7.csv', 'spd3vis13sep9.csv', 'spd3vis17sep1.csv', 'spd3vis17sep3.csv', 'spd3vis17sep5.csv', 'sp

In [156]:
df.head()

,Step,AgentID,NormDir,NeighborInfo,run
0,1,1,[0.29104821 0.0766334 ],"[0.1, 0.05, 0.1, 0.36751082894187825, 0.096766...",0
1,1,2,[-0.99886693 0.04759056],"[0.1, 0.05, 0.1, -0.9478251831935098, 0.088954...",0
2,1,3,[-0.59417386 0.83265556],"[0.1, 0.05, 0.1, -0.6872096294683163, 0.963032...",0
3,1,4,[ 0.33599196 -1.6030588 ],"[0.1, 0.05, 0.1, 0.10275643221227515, -0.49026...",0
4,1,5,[ 0.22106191 -0.54878411],"[0.1, 0.05, 0.1, 0.22106190979533058, -0.54878...",0


## Prepare Data

In [157]:
# df["NeighborDiffSum"] = df["NeighborDiffSum"].apply(lambda x: eval(x) if isinstance(x, str) else x)
df["NormDir"] = df["NormDir"].apply(lambda s: list(map(float, s.strip('[]').split())))
df["NeighborInfo"] = df["NeighborInfo"].apply(lambda x: eval(x) if isinstance(x, str) else x)



In [158]:

"""
group data according to each time step
"""
# grouped = df.groupby("Step").agg(list)
# neighbor_diff_sum = np.array(grouped["NeighborDiffSum"].to_list())  # Shape: (num_steps, num_agents, 2)
neighbor_info = np.array(df["NeighborInfo"].to_list())  # Shape: (num_steps, num_agents, 2)
normdir = np.array(df["NormDir"].to_list()) #.reshape(len(grouped), len(grouped.iloc[0]["NormDir"]), 1)  # Shape: (num_steps, num_agents, 1)

"""
this makes sure angle difference wraps correctly within [-pi. pi]
"""
# def angle_difference(theta2, theta1):
#     return np.arctan2(np.sin(theta2 - theta1), np.cos(theta2 - theta1))
# delta_angle = np.zeros_like(angles)
# delta_angle[1:] = angle_difference(angles[1:], angles[:-1]) # Shape: (num_steps, num_agents, 1)

# grouped["deltaAngle"] = delta_angle.reshape(len(grouped), -1).tolist()

# X_train, X_test, y_train, y_test = train_test_split(neighbor_diff_sum, delta_angle, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(neighbor_info, normdir, test_size=0.2, random_state=42)

# to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

## For Debugging

In [159]:
normdir.mean(axis=0)

array([-0.01657095,  0.03072082])

In [160]:
normdir.shape

(4500000, 2)

In [161]:
df['NeighborInfo'].head()

0    [0.1, 0.05, 0.1, 0.36751082894187825, 0.096766...
1    [0.1, 0.05, 0.1, -0.9478251831935098, 0.088954...
2    [0.1, 0.05, 0.1, -0.6872096294683163, 0.963032...
3    [0.1, 0.05, 0.1, 0.10275643221227515, -0.49026...
4    [0.1, 0.05, 0.1, 0.22106190979533058, -0.54878...
Name: NeighborInfo, dtype: object

In [162]:
df['NormDir'].head()

0      [0.29104821, 0.0766334]
1    [-0.99886693, 0.04759056]
2    [-0.59417386, 0.83265556]
3     [0.33599196, -1.6030588]
4    [0.22106191, -0.54878411]
Name: NormDir, dtype: object

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# # Compute incorrect delta angles (naive subtraction)
# naive_delta_angle = np.zeros_like(angles)
# naive_delta_angle[1:] = angles[1:] - angles[:-1]

# # Compute corrected delta angles (using atan2)
# corrected_delta_angle = np.zeros_like(angles)
# corrected_delta_angle[1:] = np.arctan2(np.sin(angles[1:] - angles[:-1]), np.cos(angles[1:] - angles[:-1]))

# # Plot angles over time
# plt.figure(figsize=(12, 6))

# plt.subplot(3, 1, 1)
# plt.plot(angles[:, 3, 0], label="Angle (Agent 1)")
# plt.xlabel("Time Step")
# plt.ylabel("Angle (radians)")
# plt.title("Angle Over Time")
# plt.legend()

# plt.subplot(3, 1, 2)
# plt.plot(naive_delta_angle[:, 1, 0], label="Naive Delta Angle", color="red")
# plt.xlabel("Time Step")
# plt.ylabel("Δ Angle")
# plt.title("Naive Delta Angle (No Wrap Handling)")
# plt.legend()

# plt.subplot(3, 1, 3)
# plt.plot(corrected_delta_angle[:, 1, 0], label="Corrected Delta Angle", color="green")
# plt.xlabel("Time Step")
# plt.ylabel("Δ Angle")
# plt.title("Corrected Delta Angle (With atan2 Handling)")
# plt.legend()

# plt.tight_layout()
# plt.show()


In [6]:
# normdir_df = pd.DataFrame(normdir.reshape(len(grouped), -1))  # col: agent number; row: timestep
# delta_angle_df = pd.DataFrame(delta_angle.reshape(len(grouped), -1))  # Each column is an agent's deltaAngle

NameError: name 'grouped' is not defined

In [ ]:
# normdir_df = pd.DataFrame(

In [ ]:
# normdir_df.head(30)

In [ ]:
# delta_angle_df.head(10)

In [ ]:
np.arctan2(-0.09105, 0.9958)

## Define Model

In [163]:
# define nn
hidden_size_1 = 20
hidden_size_2 = 8
hidden_size_3 = 4

model = nn.Sequential(
    nn.Linear(17, hidden_size_1),
    nn.ReLU(),
    nn.Linear(hidden_size_1, hidden_size_1),
    nn.ReLU(),
    nn.Linear(hidden_size_1, hidden_size_2),
    nn.ReLU(),
    nn.Linear(hidden_size_2, 2)
)

# define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Define Model

In [167]:
num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor.view(-1, 17))
    loss = criterion(outputs, y_train_tensor.view(-1, 2))
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}')

Epoch [10/1000], Loss: 3.842391
Epoch [20/1000], Loss: 3.841968
Epoch [30/1000], Loss: 3.841324
Epoch [40/1000], Loss: 3.841191
Epoch [50/1000], Loss: 3.840996
Epoch [60/1000], Loss: 3.840882
Epoch [70/1000], Loss: 3.840280
Epoch [80/1000], Loss: 3.839952
Epoch [90/1000], Loss: 3.839534
Epoch [100/1000], Loss: 3.839298
Epoch [110/1000], Loss: 3.838818
Epoch [120/1000], Loss: 3.838433
Epoch [130/1000], Loss: 3.838221
Epoch [140/1000], Loss: 3.838048
Epoch [150/1000], Loss: 3.837396
Epoch [160/1000], Loss: 3.836944
Epoch [170/1000], Loss: 3.836641
Epoch [180/1000], Loss: 3.836246
Epoch [190/1000], Loss: 3.835889
Epoch [200/1000], Loss: 3.835588
Epoch [210/1000], Loss: 3.835276
Epoch [220/1000], Loss: 3.834566
Epoch [230/1000], Loss: 3.834436
Epoch [240/1000], Loss: 3.834122
Epoch [250/1000], Loss: 3.833721
Epoch [260/1000], Loss: 3.833416
Epoch [270/1000], Loss: 3.833189
Epoch [280/1000], Loss: 3.832664
Epoch [290/1000], Loss: 3.832120
Epoch [300/1000], Loss: 3.831636
Epoch [310/1000], L

## Evaluate the model

In [169]:
with torch.no_grad():
    y_pred = model(X_test_tensor.view(-1, 17))
    test_loss = criterion(y_pred, y_test_tensor.view(-1, 2))
    print(f'Test Loss: {test_loss.item():.6f}')
    print(y_pred[0], y_test_tensor.view(-1, 2)[0], X_test_tensor.view(-1, 17)[0][3:5])
    print(y_pred[1], y_test_tensor.view(-1, 2)[1], X_test_tensor.view(-1, 17)[1][3:5])
    print(y_pred[2], y_test_tensor.view(-1, 2)[2], X_test_tensor.view(-1, 17)[2][3:5])
    print(y_pred[3], y_test_tensor.view(-1, 2)[3], X_test_tensor.view(-1, 17)[3][3:5])
    print(y_pred[4], y_test_tensor.view(-1, 2)[4], X_test_tensor.view(-1, 17)[4][3:5])

Test Loss: 3.403774
tensor([ 1.0166, -0.5935]) tensor([ 0.9748, -0.2233]) tensor([ 0.8611, -0.5084])
tensor([0.0129, 1.1023]) tensor([0.5402, 1.7124]) tensor([-0.3356,  0.9420])
tensor([-0.8896, -0.3172]) tensor([-1.4264, -0.4759]) tensor([-0.9877, -0.1563])
tensor([-1.0456,  0.4186]) tensor([-1.6805,  0.6251]) tensor([-0.9579,  0.2872])
tensor([-0.4498,  0.7818]) tensor([-3.8563,  3.2109]) tensor([-0.5651,  0.8250])


## Save the model

In [170]:
torch.save(model, '../models/20250420_v2')